In [62]:
# !pip install beutifulsoup4
# !pip install selenium
# !pip install tabulate

ERROR: Could not find a version that satisfies the requirement beutifulsoup4 (from versions: none)
ERROR: No matching distribution found for beutifulsoup4


In [16]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import urllib
import time
from tabulate import tabulate

In [14]:
print("test")

test


In [17]:
#create a webdriver object and set options for headless browsing
options = Options()
options.headless = True
driver = webdriver.Chrome('./chromedriver',options=options)


In [18]:
def get_js_soup(url,driver):
    driver.get(url)

    time.sleep(5)

    redirectedUrl = driver.current_url

    if url != redirectedUrl:
        print("redirected URL : " + redirectedUrl)
        newUrl = redirectedUrl + "price-target/"
        print("new URL for Analysts rating : " + newUrl)
        driver.get(url)
    else:
        print("URL for Analysts rating: " + redirectedUrl)


    time.sleep(5)
    res_html = driver.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

In [19]:
''' More tidying
Sometimes the text extracted HTML webpage may contain javascript code and some style elements. 
This function removes script and style tags from HTML so that extracted text does not contain them.
'''
def remove_script(soup):
    for script in soup(["script", "style"]):
        script.decompose()
    return soup

In [20]:
def get_mb_url(market,stocksymbol):
    base_url = "https://www.marketbeat.com/stocks/{market}/{stocksymbol}/price-target/"
    final_url = base_url.format(market = market, stocksymbol = stocksymbol)
    return final_url


In [23]:
url = get_mb_url("NASDAQ", "AAPL")
soup = get_js_soup(url,driver)

URL for Analysts rating: https://www.marketbeat.com/stocks/NASDAQ/AAPL/price-target/


In [24]:
soup = remove_script(soup)

In [25]:
data = []
data_header = ["Date [MM/dd/YYYY]", "Market analyst", "Action", "Rating", "Price target", "Price impact"]

soup = remove_script(soup)
table = soup.find("table", attrs={"class":"scroll-table sort-table fixed-left-column fixed-header"})
if table is None:
    print("No table found for market analyst rating")
else:
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele.split("➝")[-1].strip() for ele in cols if ele]) 
        #print (tabulate(data,headers=data_header))




Date [MM/dd/YYYY]    Market analyst           Action             Rating    Price target    Price impact
-------------------  -----------------------  -----------------  --------  --------------  --------------
10/30/2020           The Goldman Sachs Group  Reiterated Rating  Sell      $75.00          Medium
Date [MM/dd/YYYY]    Market analyst           Action              Rating    Price target    Price impact
-------------------  -----------------------  ------------------  --------  --------------  --------------
10/30/2020           The Goldman Sachs Group  Reiterated Rating   Sell      $75.00          Medium
10/30/2020           DA Davidson              Boost Price Target  Buy       $133.00         Medium
Date [MM/dd/YYYY]    Market analyst           Action              Rating      Price target    Price impact
-------------------  -----------------------  ------------------  ----------  --------------  --------------
10/30/2020           The Goldman Sachs Group  Reiterated Rating   

In [26]:
import pandas as pd

In [46]:
print("total no of rows: ", df.shape)
df1 = df.dropna()
print("total no of rows: ", df1.shape)

df = pd.DataFrame(data)
df.columns = data_header

df['Date'] = pd.to_datetime(df['Date [MM/dd/YYYY]'],
                                format="%m/%d/%Y",
                                errors='coerce')

print(df[0:7])

total no of rows:  (56, 7)
total no of rows:  (44, 7)
  Date [MM/dd/YYYY]           Market analyst              Action      Rating  \
0        10/30/2020  The Goldman Sachs Group   Reiterated Rating        Sell   
1        10/30/2020              DA Davidson  Boost Price Target         Buy   
2        10/30/2020            Raymond James  Boost Price Target  Outperform   
3        10/30/2020                UBS Group    Set Price Target     Neutral   
4        10/30/2020                 Barclays    Set Price Target     Neutral   
5        10/26/2020      Credit Suisse Group    Set Price Target     Neutral   
6        10/26/2020             Loop Capital  Lower Price Target        Hold   

  Price target Price impact       Date  
0       $75.00       Medium 2020-10-30  
1      $133.00       Medium 2020-10-30  
2      $140.00         High 2020-10-30  
3      $115.00         High 2020-10-30  
4      $100.00       Medium 2020-10-30  
5      $106.00          Low 2020-10-26  
6      $135.00    

In [52]:
from datetime import datetime


current_month = datetime.now().strftime("%Y-%m")

print("Current month", current_month)


Current month 2020-10


In [56]:
df3 = df.dropna()
current_month_mask = df3['Date'].map(lambda x: x.strftime("%Y-%m")) == current_month
df4 = df3[current_month_mask].sort_values(by=['Date'], ascending=False)

print(df4)

   Date [MM/dd/YYYY]             Market analyst              Action  \
0         10/30/2020    The Goldman Sachs Group   Reiterated Rating   
1         10/30/2020                DA Davidson  Boost Price Target   
2         10/30/2020              Raymond James  Boost Price Target   
3         10/30/2020                  UBS Group    Set Price Target   
4         10/30/2020                   Barclays    Set Price Target   
5         10/26/2020        Credit Suisse Group    Set Price Target   
6         10/26/2020               Loop Capital  Lower Price Target   
7         10/26/2020        Atlantic Securities  Initiated Coverage   
8         10/26/2020       JPMorgan Chase & Co.    Set Price Target   
9         10/22/2020              Piper Sandler             Upgrade   
11        10/14/2020                    Wedbush  Boost Price Target   
13        10/14/2020  Jefferies Financial Group  Boost Price Target   
14        10/14/2020                      Argus             Upgrade   
15    

In [61]:
dropped_columns = ["Date [MM/dd/YYYY]", "Action", "Price target", "Price impact"]

df5 = df4.drop(columns=dropped_columns)

json_obj = df5.to_json(orient='records', date_format='iso')
print(json_obj)

[{"Market analyst":"The Goldman Sachs Group","Rating":"Sell","Date":"2020-10-30T00:00:00.000Z"},{"Market analyst":"DA Davidson","Rating":"Buy","Date":"2020-10-30T00:00:00.000Z"},{"Market analyst":"Raymond James","Rating":"Outperform","Date":"2020-10-30T00:00:00.000Z"},{"Market analyst":"UBS Group","Rating":"Neutral","Date":"2020-10-30T00:00:00.000Z"},{"Market analyst":"Barclays","Rating":"Neutral","Date":"2020-10-30T00:00:00.000Z"},{"Market analyst":"Credit Suisse Group","Rating":"Neutral","Date":"2020-10-26T00:00:00.000Z"},{"Market analyst":"Loop Capital","Rating":"Hold","Date":"2020-10-26T00:00:00.000Z"},{"Market analyst":"Atlantic Securities","Rating":"Overweight","Date":"2020-10-26T00:00:00.000Z"},{"Market analyst":"JPMorgan Chase & Co.","Rating":"Buy","Date":"2020-10-26T00:00:00.000Z"},{"Market analyst":"Piper Sandler","Rating":"Overweight","Date":"2020-10-22T00:00:00.000Z"},{"Market analyst":"Wedbush","Rating":"Overweight","Date":"2020-10-14T00:00:00.000Z"},{"Market analyst":"Jef

In [67]:
!pip freeze | grep beautifulsoup4

beautifulsoup4==4.9.3
